# Convolutional_neural_network

**pytorch conv2d documentation**:  
torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')

In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [3]:
# device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [4]:
# Hyper parameters
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

In [5]:
# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../../data/',
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data/',
                                          train=False,
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)


Extracting ../../data/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/MNIST/raw




Extracting ../../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/MNIST/raw




Extracting ../../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/MNIST/raw




Extracting ../../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [9]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc = nn.Linear(7*7*32, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

In [10]:
model = ConvNet(num_classes).to(device)

In [11]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [12]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
          print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(
              epoch+1, num_epochs, i+1, total_step, loss.item()
          ))

Epoch [1/5], Step [100/600], Loss: 0.2019
Epoch [1/5], Step [200/600], Loss: 0.1347
Epoch [1/5], Step [300/600], Loss: 0.0586
Epoch [1/5], Step [400/600], Loss: 0.0486
Epoch [1/5], Step [500/600], Loss: 0.0528
Epoch [1/5], Step [600/600], Loss: 0.0586
Epoch [2/5], Step [100/600], Loss: 0.0095
Epoch [2/5], Step [200/600], Loss: 0.0301
Epoch [2/5], Step [300/600], Loss: 0.0066
Epoch [2/5], Step [400/600], Loss: 0.0248
Epoch [2/5], Step [500/600], Loss: 0.0570
Epoch [2/5], Step [600/600], Loss: 0.1108
Epoch [3/5], Step [100/600], Loss: 0.0753
Epoch [3/5], Step [200/600], Loss: 0.0630
Epoch [3/5], Step [300/600], Loss: 0.0119
Epoch [3/5], Step [400/600], Loss: 0.0657
Epoch [3/5], Step [500/600], Loss: 0.0361
Epoch [3/5], Step [600/600], Loss: 0.0383
Epoch [4/5], Step [100/600], Loss: 0.0063
Epoch [4/5], Step [200/600], Loss: 0.0103
Epoch [4/5], Step [300/600], Loss: 0.0454
Epoch [4/5], Step [400/600], Loss: 0.0180
Epoch [4/5], Step [500/600], Loss: 0.0130
Epoch [4/5], Step [600/600], Loss:

In [13]:
# test the model
model.eval() # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
      
    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

# save the model 
torch.save(model.state_dict(), 'model.ckpt')

Test Accuracy of the model on the 10000 test images: 99.08 %
